In [1]:
import findspark
findspark.init('/home/harkirat/spark-3.3.1-bin-hadoop3')

import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('rec').getOrCreate()

22/12/27 20:25:49 WARN Utils: Your hostname, harkirat-QEMU-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 192.168.64.16 instead (on interface enp0s6)
22/12/27 20:25:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/27 20:25:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
df=spark.read.csv('/home/harkirat/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Recommender_Systems/movielens_ratings.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [5]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [6]:
training, test = df.randomSplit([0.7, 0.3])

In [7]:
als=ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [8]:
model=als.fit(training)

In [9]:
predictions=model.transform(test)

In [10]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      2|   4.0|    28| -1.6207302|
|      2|   1.0|    26| -0.6671209|
|      2|   1.0|    12|  3.7919033|
|      0|   1.0|    22|  1.5887235|
|      4|   2.0|     1|  0.9179796|
|      0|   1.0|     6| 0.21072254|
|      2|   1.0|    16|  1.6829898|
|      4|   2.0|    20|-0.14367247|
|      1|   1.0|     5| 0.88820773|
|      1|   1.0|    19|  2.2526336|
|      0|   1.0|    15|  1.4556341|
|      2|   1.0|    15|  1.9295807|
|      0|   1.0|     8|  2.7426124|
|      2|   4.0|     8|  3.7658021|
|      4|   1.0|     7|  1.9146041|
|      4|   3.0|    10|  1.4028716|
|      3|   1.0|    29|  1.3152908|
|      1|   1.0|    14|  0.5414072|
|      4|   3.0|     2| -1.2609648|
|      2|   3.0|     0|  1.1932188|
+-------+------+------+-----------+
only showing top 20 rows



In [11]:
evaluator=RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [13]:
rmse=evaluator.evaluate(predictions)

In [14]:
rmse

2.138349853904093

In [19]:
single_user=test.filter(test['userId']==11).select(['movieId','userId'])

In [20]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     10|    11|
|     13|    11|
|     19|    11|
|     25|    11|
|     27|    11|
|     35|    11|
|     41|    11|
|     61|    11|
|     67|    11|
|     79|    11|
|     80|    11|
|     86|    11|
|     89|    11|
|     97|    11|
|     99|    11|
+-------+------+



In [21]:
rec_user=model.transform(single_user)

In [22]:
rec_user.show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|      9|    11|   2.559655|
|     10|    11| 0.05805421|
|     13|    11|   3.503457|
|     19|    11|   2.949606|
|     25|    11|-0.03281629|
|     27|    11|  2.8182104|
|     35|    11| -0.1941858|
|     41|    11|0.040638193|
|     61|    11| 0.54365045|
|     67|    11|   0.943252|
|     79|    11|  0.7000865|
|     80|    11|  2.9098005|
|     86|    11| 0.30706918|
|     89|    11|  2.9400272|
|     97|    11| -0.3937651|
|     99|    11|  0.6093909|
+-------+------+-----------+

